Your code can subclass built-ins such as `list` or `dict`, however, there is a major caveat: the code of the built-ins does not call special methods overridden by user-defined classes. For example overridden `__getitem__()` in a subclass of `dict` will not be called by the built-in `get()` method.

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
dd = DoppelDict(one=1)
print(dd)

{'one': 1}


The `__init__` method clearly ignored the overriden `__setitem__`

In [3]:
dd['two'] = 2
print(dd)

{'one': 1, 'two': [2, 2]}


In [4]:
dd.update(three=3)
print(dd)

{'one': 1, 'two': [2, 2], 'three': 3}


The `update` method from `dict` does not use the overridden version of `__setitem__`. This stands in clear violation of the basic rule of object oriented design: the search for methods should always start from the class of the target instance, even when the call happens inside a method implemented in a superclass.

In [5]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [6]:
ad = AnswerDict(a='foo')
ad['a']

42

In [7]:
d = {}
d.update(ad)
d['a']

'foo'

Subclassing from built-in types is error prone because the built-in methods mostly ignore user-defined overrides. Instead, derive your classes from `UserDict`, `UserList` and `UserString` from the `collections` module.

In [8]:
import collections

class DoppelDict(collections.UserDict):
    
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [9]:
dd = DoppelDict(one=1)
print(dd)

{'one': [1, 1]}


In [10]:
dd.update(three=3)
print(dd)

{'one': [1, 1], 'three': [3, 3]}


The problem applies to method delegation within the C language implementation of the built-in types, and only affects user-defines classes derived directly from those types. If you subclass from a class coded in Python, such as UserDict you avoid this issues.